# The electrostatic potential of free atoms

In [1]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import numpy as np
import scipy as sc
from parse_density_files import CUBE
import alchemy_tools2 as at
import qml_interface as qmi
import utils_qm as uq
import glob
from matplotlib import pyplot as plt

from ase.units import Bohr

In [2]:
def get_paths():
    pass

def load_cube_data(paths_cubes):
    """
    returns the data necessary to calculate the atomic energies from the cube-files
    for different lambda values
    
    paths_cubes: paths to cubes files
    densities: densities given in different cube files
    lam_vals: lambda value for cube file, obtained by parsing filename
    nuclei: charges and coordinates of the nuclei
    gpts: the gridpoints where density values are given
    """
    
    densities = []
    nuclei = None # nuclear charges and their positions
    gpts = None # gridpoints where density values are given
    h_matrix = np.zeros((3,3)) # needed for the calculation of the distance of the nuclei to the gridpoints with MIC
    
    for idx, path in enumerate(paths_cubes):
        cube = CUBE(path)
        
        densities.append(cube.data_scaled) # density
        
        if idx==len(paths_cubes)-1:
            nuclei = cube.atoms
            gpts = cube.get_grid()
            h_matrix = [cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ]
    
    return(densities, nuclei, gpts, h_matrix)

def get_alchpot(density, nuc_pos, meshgrid, h_matrix):
    dist = at.distance_MIC2(nuc_pos, meshgrid, h_matrix)
    alch_pot = -(density/dist).sum()
    return(alch_pot)

def generate_ueg(num_elec, shape):
    volume = 1
    for s in shape:
        volume *= s
    return(np.full(shape, num_elec/volume))

### Maximum distance between two atoms in a molecule

In [ ]:
def get_largest_distance(coords):
    distances = sc.spatial.distance.cdist(coords, coords)
    max_dist = []
    for d in distances:
        max_dist.append(np.max(d))
    return(np.amax(max_dist))

In [ ]:
paths = qmi.wrapper_alch_data()
data, msize = qmi.load_alchemy_data(paths)

largest_dist = []
for mol in data:
    largest_dist.append(get_largest_distance(mol[:,1:4]))

In [ ]:
np.amax(largest_dist)

In [ ]:
np.linalg.norm( np.array([20,20,20])-np.array([10,10,10]) )

### Load and prepare data

In [ ]:
# #################################
# #          HYDROGEN
# #################################
# # load paths to H
# paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/*/DENSITY.cube')
# paths.append(paths[0])
# del(paths[0])
# # generate lam_vals H
# lam_vals = np.array([0.0, 23/38, 0.7, 30/38, 1])

# #################################
# #          CARBON
# #################################
# # load paths to C
# paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/C/*/DENSITY.cube')
# paths.append(paths[0])
# del(paths[0])
# # generate lam_vals C
# lam_vals = np.array([0.0, 8/38, 30/38, 1])

# #################################
# #          NITROGEN
# #################################
# # load paths to N
# paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/N/*/DENSITY.cube')
# paths.append(paths[0])
# del(paths[0])
# # generate lam_vals N
# lam_vals = np.array([0.0, 15/38, 1])

#################################
#          OXYGEN
#################################
# load paths to O
paths = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_0p2/DENSITY.cube',
'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_0p4/DENSITY.cube',
'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_0p8/DENSITY.cube',
'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_1/DENSITY.cube']
# generate lam_vals O
lam_vals = np.array([0.0, 8/38, 15/38, 30/38, 1])


In [ ]:
# load cube files
densities, nuclei, gpts, h_matrix = load_cube_data(paths)
shape = densities[0].shape

# # generate cube file for ueg H
# ueg = generate_ueg(1, shape)

# # generate cube file for ueg C
# ueg = generate_ueg(4, shape)

# generate cube file for ueg N
ueg = generate_ueg(5, shape)

# # generate cube file for ueg O
# ueg = generate_ueg(6, shape)


densities.insert(0, ueg)

### Calculate averaged density

In [ ]:
# calculate averaged density
average_density = at.integrate_lambda_density(densities, lam_vals)

### Calculate electrostatic potential
\begin{equation}
\int d\vec{r} \frac{\tilde{\rho}_{J, \text{free}}}{|\vec{r}-\vec{R}_I|}
\end{equation}
for different positions $\vec{R}_I$

In [ ]:
# prepare positions R_I
# dist1 = np.arange(0, 5, 0.25)
# dist2 = np.arange(5.5, 17.5/Bohr, 1.0)
# dist = np.concatenate((dist1, dist2))
dist = np.arange(0, 17.5/Bohr, 0.25)
dvecs = dist/np.sqrt(3)
center = 10.0/Bohr
pos_R_I = []

for dvec in dvecs:
    pos_R_I.append([center+dvec, center+dvec, center+dvec])
pos_R_I = np.array(pos_R_I)

# calculate integrals
integrals = []
for pos in pos_R_I:
    integrals.append(get_alchpot(average_density, pos, gpts, h_matrix))


In [ ]:
# fit potential
poly_obj = sc.interpolate.CubicSpline(dist, integrals)
eval_x = np.arange(0, 21/Bohr, 0.1)
integral_fit = poly_obj.__call__(eval_x)

In [ ]:
fig, ax = plt.subplots(1,1)
plt.rcParams['font.size'] = 20
ax.plot(dist, integrals, '-o')
ax.plot(eval_x, integral_fit)
ax.set_xlabel(r'$|\vec{R}_J - \vec{R}_I|$ (Bohr)')
ax.set_ylabel(r'$\int d\vec{r} \frac{\tilde{\rho}_{J, \rm{at}}}{|\vec{r}-\vec{R}_I|}$')

In [ ]:
# save result
save_path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/'
uq.save_obj(poly_obj, save_path+'poly_O_fine')

# The atomic alchemical binding potential

In [1]:
import numpy as np
from ase.units import Bohr

import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import numpy as np
import qml_interface as qmi
import utils_qm as uq
import geometry_euston as ge
import glob

In [2]:
def get_alch_bind_pot(molecule, poly_obs):
    """
    calculates the alchemical binding potential for every atom in molecule
    """
    # construct h matrix
    h_mat = ge.abc_to_hmatrix(20,20,20,90,90,90)
    # get alchpot of free atoms
    alch_pot_free = []
    for atom_I in molecule:
        alch_pot_free_I = 0
        for atom_J in molecule:
            dist = ge.distance_pbc(atom_I[1:4], atom_J[1:4], h_mat)/Bohr
            nuc_charge = atom_J[0]
            poly_obj = poly_obs[nuc_charge] # dictionary
            elec_pot = poly_obj.__call__([dist])[0]
            alch_pot_free_I += elec_pot
        alch_pot_free.append(alch_pot_free_I)
    alch_pot_free = np.array(alch_pot_free)
    alch_pot_bind = molecule[:, 4] - alch_pot_free
    return(alch_pot_bind)

def get_element_charge(el):
    if el == 'H':
        return(1)
    elif el == 'C':
        return(6)
    elif el == 'N':
        return(7)
    elif el == 'O':
        return(8)
    else:
        raise ValueError('Symbol for given charge not available')

def load_poly_objs(paths_poly):
    poly_objs = dict()
    for p in paths_poly:
        el = p.split('/')[-1].split('_')[-1]
        charge = get_element_charge(el)
        poly_objs[charge] = uq.load_obj(p)
    return(poly_objs)

In [3]:
# load molecule data and electrostatic potentials
paths = qmi.wrapper_alch_data()
basepath = [p.strip('atomic_energies_with_mic.txt') for p in paths]
data, msize = qmi.load_alchemy_data(paths)

# paths_poly = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/*/poly_*_fine')
# poly_objs = load_poly_objs(paths_poly)
paths_poly = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/poly_H_fine',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/C/poly_C_fine',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/N/poly_N_fine',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/poly_O_fine']

poly_objs = dict()
for k,p in zip([1.0, 6.0, 7.0, 8.0], paths_poly):
    poly_objs[k] = uq.load_obj(p)

In [4]:
# calculate potentials
for molecule, path in zip(data, basepath):
    # calculate binding values
    alch_pot_bind = get_alch_bind_pot(molecule, poly_objs)
    atomic_atomisation = alch_pot_bind*molecule[:,0]
    # save data
    save_arr = np.concatenate( (molecule[:,0:5], np.array([alch_pot_bind]).T, np.array([atomic_atomisation]).T), axis=1)
    h = 'charge\t x_coord\t y_coord\t z_coord\t alchemical_potential\t alchemical_binding_potential\t atomisation_energies'
    np.savetxt(path+'atomic_binding_energies.txt', save_arr, delimiter='\t', header=h)